In [2]:
import keras
import pickle
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
import os
import numpy as np
import glob
import random
import math
from keras.callbacks import ModelCheckpoint
import time
import keras.backend as K
import tensorflow as tf
from tensorflow import set_random_seed
import subprocess
from collections import Counter
import socket
import sys

Using TensorFlow backend.


In [3]:
argvv = ['./test']
seed_list = glob.glob('./neuzz_in/*')
seed_list.sort()
SPLIT_RATIO = len(seed_list)
rand_index = np.arange(SPLIT_RATIO)
np.random.shuffle(seed_list)

In [4]:
MAX_FILE_SIZE = 512
call=subprocess.check_output
raw_bitmap = {}
tmp_cnt = []
out = ''
for f in seed_list:
    tmp_list = []
    try:
        # append "-o tmp_file" to strip's arguments to avoid tampering tested binary.
        print(f)
        with open(f) as myinput:
            out = call(['./afl-showmap', '-q', '-e', '-o', '/dev/stdout', '-m', '512'] + argvv, stdin=myinput)
    except subprocess.CalledProcessError:
        print("find a crash")
    for line in out.splitlines():
        edge = line.split(':')[0]
        tmp_cnt.append(edge)
        tmp_list.append(edge)
    raw_bitmap[f] = tmp_list

./neuzz_in/id:000041,src:000040,op:int8,pos:17,val:+0,+cov
./neuzz_in/id:000003,src:000000,op:int8,pos:0,val:+0,+cov
./neuzz_in/id:000019,src:000017,op:arith8,pos:6,val:+13,+cov
./neuzz_in/id:000099,src:000097,op:flip2,pos:46,+cov
./neuzz_in/id:000124,src:000121,op:int8,pos:58,val:+0,+cov
./neuzz_in/id:000067,src:000065,op:flip2,pos:30,+cov
./neuzz_in/id:000082,src:000080,op:int8,pos:37,val:+0,+cov
./neuzz_in/id:000013,src:000012,op:flip1,pos:3,+cov
./neuzz_in/id:000045,src:000043,op:arith8,pos:19,val:+21,+cov
./neuzz_in/id:000070,src:000067,op:int8,pos:31,val:+0,+cov
./neuzz_in/id:000072,src:000069,op:havoc,rep:2,+cov
./neuzz_in/id:000061,src:000059,op:flip4,pos:27,+cov
./neuzz_in/id:000069,src:000067,op:flip4,pos:31,+cov
./neuzz_in/id:000062,src:000059,op:int8,pos:27,val:+0,+cov
./neuzz_in/id:000011,src:000010,op:flip1,pos:2,+cov
./neuzz_in/id:000133,src:000131,op:arith8,pos:63,val:+11,+cov
./neuzz_in/id:000064,src:000062,op:havoc,rep:8,+cov
./neuzz_in/id:000015,src:000013,op:havoc,r

In [5]:
counter = Counter(tmp_cnt).most_common()
label = [int(f[0]) for f in counter]
bitmap = np.zeros((len(seed_list), len(label)))
for idx,i in enumerate(seed_list):
    tmp = raw_bitmap[i]
    for j in tmp:
        if int(j) in label:
            bitmap[idx][label.index((int(j)))] = 1

In [6]:
if os.path.isdir("./bitmaps/") == False:
    os.makedirs('./bitmaps')
fit_bitmap = np.unique(bitmap,axis=1)
MAX_BITMAP_SIZE = fit_bitmap.shape[1]
for idx,i in enumerate(seed_list):
    file_name = "./bitmaps/"+i.split('/')[-1]
    np.save(file_name,fit_bitmap[idx])

In [8]:
fit_label = []
for i in range(fit_bitmap.shape[1]):
    edges = []
    for j in range(bitmap.shape[1]):
        if (bitmap[:, j] == fit_bitmap[:, i]).all():
            edges.append(j)
    fit_label.append(edges)

In [9]:
def getIndex(edge):
    index = label.index(edge)
    for i, edges in enumerate(fit_label):
        if index in edges:
            return i
    return None

In [10]:
def accur_1(y_true,y_pred):
    y_true = tf.round(y_true)
    pred = tf.round(y_pred)
    summ = tf.constant(MAX_BITMAP_SIZE,dtype=tf.float32)
    wrong_num = tf.subtract(summ,tf.reduce_sum(tf.cast(tf.equal(y_true, pred),tf.float32),axis=-1))
    right_1_num = tf.reduce_sum(tf.cast(tf.logical_and(tf.cast(y_true,tf.bool), tf.cast(pred,tf.bool)),tf.float32),axis=-1)
    ret = K.mean(tf.divide(right_1_num,tf.add(right_1_num,wrong_num)))
    return ret

In [11]:
def build_model():
    batch_size = 32
    num_classes = MAX_BITMAP_SIZE
    epochs = 50
    model = Sequential()
    model.add(Dense(4096, input_dim=MAX_FILE_SIZE))
    model.add(Activation('relu'))
    model.add(Dense(num_classes))
    model.add(Activation('sigmoid'))
    opt = keras.optimizers.adam(lr=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[accur_1])
    model.summary()
    return model

In [12]:
model = build_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4096)              2101248   
_________________________________________________________________
activation_1 (Activation)    (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 258)               1057026   
_________________________________________________________________
activation_2 (Activation)    (None, 258)               0         
Total params: 3,158,274
Trainable params: 3,158,274
Non-trainable params: 0
_________________________________________________________________


In [13]:
def generate_training_data(lb,ub):
    seed = np.zeros((ub-lb,MAX_FILE_SIZE))
    bitmap = np.zeros((ub-lb,MAX_BITMAP_SIZE))
    for i in range(lb,ub):
        tmp = open(seed_list[i],'r').read()
        ln = len(tmp)
        if ln < MAX_FILE_SIZE:
            tmp = tmp + (MAX_FILE_SIZE - ln) * '\0'
        seed[i-lb] = [ord(j) for j in list(tmp)]
    for i in range(lb,ub):
        file_name = "./bitmaps/"+ seed_list[i].split('/')[-1] + ".npy"
        bitmap[i-lb] = np.load(file_name)
    return seed,bitmap

In [14]:
def train_generate(batch_size):
    global seed_list
    while 1:
        np.random.shuffle(seed_list)
        for i in range(0,SPLIT_RATIO,batch_size):
            if (i+batch_size) > SPLIT_RATIO:
                x,y=generate_training_data(i,SPLIT_RATIO)
                x = x.astype('float32')/255
            else:
                x,y=generate_training_data(i,i+batch_size)
                x = x.astype('float32')/255
            yield (x,y)

In [15]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.lr = []
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(step_decay(len(self.losses)))
        print(step_decay(len(self.losses)))

In [16]:
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.7
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop,math.floor((1+epoch)/epochs_drop))
    return lrate

In [17]:
def train(model):
    loss_history = LossHistory()
    lrate = keras.callbacks.LearningRateScheduler(step_decay)
    callbacks_list = [loss_history, lrate]
    model.fit_generator(train_generate(8),
              steps_per_epoch = (SPLIT_RATIO/8 + 1),
              epochs=100,
              verbose=1, callbacks=callbacks_list)
    # Save model and weights
    model.save_weights("hard_label.h5")

In [18]:
train(model)

Epoch 1/100
17/17 [==============================] - 1s 64ms/step - loss: 0.3900 - accur_1: 0.5973
0.001
Epoch 2/100
17/17 [==============================] - 1s 32ms/step - loss: 0.2079 - accur_1: 0.7131
0.001
Epoch 3/100
17/17 [==============================] - 1s 32ms/step - loss: 0.1619 - accur_1: 0.7604
0.001
Epoch 4/100
17/17 [==============================] - 1s 35ms/step - loss: 0.1419 - accur_1: 0.7628
0.001
Epoch 5/100
17/17 [==============================] - 1s 35ms/step - loss: 0.1341 - accur_1: 0.7894
0.001
Epoch 6/100
17/17 [==============================] - 1s 37ms/step - loss: 0.1277 - accur_1: 0.7837
0.001
Epoch 7/100
17/17 [==============================] - 1s 36ms/step - loss: 0.1199 - accur_1: 0.7977
0.001
Epoch 8/100
17/17 [==============================] - 1s 36ms/step - loss: 0.1207 - accur_1: 0.7910
0.001
Epoch 9/100
17/17 [==============================] - 1s 35ms/step - loss: 0.1110 - accur_1: 0.8130
0.0007
Epoch 10/100
17/17 [==============================] - 

17/17 [==============================] - 1s 43ms/step - loss: 0.0570 - accur_1: 0.8935
8.23543e-05
Epoch 77/100
17/17 [==============================] - 1s 41ms/step - loss: 0.0569 - accur_1: 0.8945
8.23543e-05
Epoch 78/100
17/17 [==============================] - 1s 39ms/step - loss: 0.0570 - accur_1: 0.8943
8.23543e-05
Epoch 79/100
17/17 [==============================] - 1s 42ms/step - loss: 0.0567 - accur_1: 0.8947
5.764801e-05
Epoch 80/100
17/17 [==============================] - 1s 40ms/step - loss: 0.0566 - accur_1: 0.8949
5.764801e-05
Epoch 81/100
17/17 [==============================] - 1s 43ms/step - loss: 0.0566 - accur_1: 0.8942
5.764801e-05
Epoch 82/100
17/17 [==============================] - 1s 40ms/step - loss: 0.0569 - accur_1: 0.8938
5.764801e-05
Epoch 83/100
17/17 [==============================] - 1s 42ms/step - loss: 0.0568 - accur_1: 0.8963
5.764801e-05
Epoch 84/100
17/17 [==============================] - 1s 42ms/step - loss: 0.0564 - accur_1: 0.8955
5.764801e-05

In [19]:
def vectorize_file(fl, isfile, vectorize=True):
    seed = np.zeros((1,MAX_FILE_SIZE))
    if isfile:
        tmp = open(fl,'r').read()
    else:
        tmp = fl
    ln = len(tmp)
    if ln < MAX_FILE_SIZE:
        tmp = tmp + (MAX_FILE_SIZE - ln) * '\0'
    seed[0] = [ord(j) for j in list(tmp)]
    if vectorize:
        seed = seed.astype('float32')/255
    return seed

In [20]:
def gradient(model, edge, seed, isfile, vectorize=True):
    layer_list = [(layer.name, layer) for layer in model.layers]
    index = getIndex(edge)
    loss = layer_list[-2][1].output[:,index]
    grads = K.gradients(loss,model.input)[0]
    iterate = K.function([model.input], [loss, grads])
    x=vectorize_file(seed, isfile, vectorize)
    loss_value, grads_value = iterate([x])
    idx = np.flip(np.argsort(np.absolute(grads_value),axis=1)[:, -MAX_FILE_SIZE:].reshape((MAX_FILE_SIZE,)),0)
    val = np.sign(grads_value[0][idx])
    return idx, val, grads_value

In [65]:
idx, val, grads = gradient(model, 65010, "xhqqevmyaua", False, True)
print(idx[:32])
for i in range(32):
    print(idx[i], grads.reshape((MAX_FILE_SIZE,))[idx[i]])

[ 9 19  7 15 33 30 28 34 14 16  8  2 31 32 35  5 13  3 10 58 18 20  4  6
 27 23 39 60 38 56 40 21]
(9, 7.1797113)
(19, 5.3130484)
(7, 4.081481)
(15, 3.6039388)
(33, 3.3900166)
(30, -3.360676)
(28, -3.2450254)
(34, 3.2305295)
(14, 3.1713207)
(16, 3.0232043)
(8, -3.0090027)
(2, 3.0035567)
(31, -2.906313)
(32, 2.7794313)
(35, 2.7166095)
(5, 2.4512334)
(13, 2.3961582)
(3, -2.0681763)
(10, 1.8895186)
(58, -1.8839817)
(18, 1.8257998)
(20, 1.7273021)
(4, -1.6383826)
(6, 1.6315525)
(27, -1.5775269)
(23, -1.5609598)
(39, 1.5214111)
(60, -1.5152252)
(38, 1.4620535)
(56, -1.4037094)
(40, 1.3544334)
(21, 1.344499)


In [46]:
idx, val, grads = gradient(model, 63838, "xhqqevmyaua", False, False)
print(idx[:32])
for i in range(32):
    print(idx[i], grads.reshape((MAX_FILE_SIZE,))[idx[i]])

[ 2  9  8  0  7 19 58 33 34 11 60 30 28 35 56 15  1  5 20 61 32 31 38  6
 39 16 55 40 21 26  4 63]
(2, 4.3056946)
(9, 3.4857693)
(8, -3.485417)
(0, 3.4091678)
(7, 3.253026)
(19, 3.1068797)
(58, -2.4985132)
(33, 2.3860934)
(34, 2.3333309)
(11, -2.0473819)
(60, -1.9499453)
(30, -1.9294356)
(28, -1.8983822)
(35, 1.8873602)
(56, -1.820421)
(15, 1.8177513)
(1, 1.6894426)
(5, 1.6128211)
(20, 1.584897)
(61, -1.5840384)
(32, 1.5515348)
(31, -1.4208286)
(38, 1.3686905)
(6, 1.3130174)
(39, 1.265341)
(16, 1.2509533)
(55, -1.2221736)
(40, 1.2094162)
(21, 1.1958565)
(26, 1.0776446)
(4, -1.0232619)
(63, -1.0156202)


In [24]:
idx, val, grads = gradient(model, 46697, "neuzz_in/id:000134,src:000131,op:int8,pos:63,val:+0,+cov", True, False)
idx

array([  2,  51,   9,   0,  58,  55,  19,  56,  45,   7,   1,   3,   5,
        53,  33,  44,  60,  15,  20,  46,   6,  21,  34,  32,  43,  54,
        50,  31,  35,  30,  42,  13,  16,  10,  57,   8,  14,  12,  24,
        26,  18,  41,  28,  49,  22,   4,  48,  17,  29,  59,  37,  38,
       338,  25, 444,  23, 390,  52,  63,  27, 223, 106, 475, 283, 103,
       389, 502, 122, 367, 466, 197, 221, 321, 494, 453, 395, 400,  75,
       378, 264,  61, 270, 414, 438, 342, 361, 246, 211,  78, 462,  72,
       316, 300, 447, 105, 377, 107, 267, 275, 192, 302, 174, 265, 249,
       402,  39, 252, 433, 263, 473, 169, 280, 156, 261, 172, 206, 164,
       124, 474, 346,  76, 157, 142, 306,  90, 196, 307, 385, 391,  74,
       345, 114, 360, 282, 369, 274, 384, 150, 297,  62, 355, 286, 320,
       133, 417, 352,  73,  84, 115, 244, 100, 335, 230, 424, 238, 173,
       226, 248, 199, 117, 193, 140, 451, 381, 147, 362, 205, 380, 498,
       309, 442, 212, 276, 358, 120, 102, 121, 153, 333, 490, 15

In [25]:
for i in range(MAX_FILE_SIZE):
    print(idx[i], grads.reshape((MAX_FILE_SIZE,))[idx[i]])

(2, -0.9926252)
(51, 0.9877852)
(9, -0.94090235)
(0, -0.9284056)
(58, 0.9225007)
(55, 0.9210491)
(19, -0.8444284)
(56, 0.78428847)
(45, 0.7777022)
(7, -0.76277465)
(1, -0.59241474)
(3, -0.519421)
(5, -0.49340814)
(53, 0.4735327)
(33, -0.4687665)
(44, 0.43126237)
(60, 0.43033266)
(15, -0.41055226)
(20, -0.40064228)
(46, 0.39519095)
(6, -0.38543844)
(21, -0.35823664)
(34, -0.35743427)
(32, -0.35670954)
(43, -0.35147437)
(54, 0.3381382)
(50, 0.3327644)
(31, 0.31581885)
(35, -0.31480682)
(30, 0.3088276)
(42, 0.30355877)
(13, -0.29671925)
(16, -0.28662738)
(10, -0.27887574)
(57, 0.27090114)
(8, 0.25896052)
(14, -0.25012976)
(12, -0.24485919)
(24, -0.2355277)
(26, -0.23170716)
(18, -0.21151908)
(41, -0.20655754)
(28, 0.18687436)
(49, -0.1807828)
(22, -0.16419214)
(4, -0.15219514)
(48, 0.13927597)
(17, -0.13549107)
(29, 0.13072295)
(59, -0.11105962)
(37, -0.10061196)
(38, 0.09978367)
(338, -0.093771756)
(25, -0.089932725)
(444, -0.089920774)
(23, 0.081211895)
(390, 0.07882668)
(52, 0.07639997

In [33]:
idx, val = gradient(model, 46697, "neuzz_in/id:000134,src:000131,op:int8,pos:63,val:+0,+cov", True, False)
idx[:32]

array([58, 51, 56, 55, 45, 60,  9,  0, 19, 33,  2, 53, 44, 59, 46, 43,  7,
       34, 42, 32, 20, 35, 50, 26,  1,  5, 49, 24, 21, 57, 41, 54])

In [36]:
def evaluate(model, seed, vectorize=False):
    x = vectorize_file(seed, True, vectorize)
    file_name = file_name = "./bitmaps/"+ seed.split('/')[-1] + ".npy"
    return model.predict(x), np.load(file_name)

In [39]:
y, ty = evaluate(model, "neuzz_in/id:000134,src:000131,op:int8,pos:63,val:+0,+cov", False)

In [40]:
y

array([[0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.00

In [41]:
ty

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.